<a href="https://colab.research.google.com/github/Aimy99/Project-Langchain-Calculator-Tool/blob/main/Project_LangChain_Calculator_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **PIAIC Quarter 3 - Project 03: 'LangChain Function/Tool Calling Calculator'**

*Developed by Aiman Baquar*

### **Overview:**

The LangChain Function/Tool Calling Calculator uses the Google Gemini model to perform real-time calculations through a conversational interface. By integrating LangChain's tool-calling feature, it dynamically responds to user queries and handles arithmetic tasks efficiently.


### **Technology Stack:**

*   **LangChain** (Integration): Acts as the bridge connecting the Google Gemini Flash model to custom tools like the calculator.

*   **Google Gemini** (AI Model): Powers natural language understanding and generates responses, invoking the calculator tool as needed to compute results.

*  **Python** (Programming Language): Implements the core logic for the calculator and integrates it with LangChain for smooth functionality.

*   **Google Colab** (Environment): Built and executed within a Google Colab notebook, leveraging cloud-based resources for enhanced scalability.

### **Outcome:**

A fully functional conversational calculator that processes arithmetic queries and performs real-time calculations. The tool efficiently computes mathematical expressions, delivering accurate and timely results in response to user input.

### **Use Cases:**

*  Educational Tools: Assists students in solving math problems interactively and in real-time.


*  Customer Support: Can be embedded into support bots to quickly handle customer queries involving calculations.


*  E-commerce: Facilitates calculations for discounts, taxes, shipping costs, and other related operations.


*  Finance: Computes financial metrics such as loan interest, ROI, or currency exchange rates.


*  Healthcare: Performs medical calculations like BMI or dosage conversions for healthcare professionals.


*  Productivity Apps: Seamlessly integrated into personal assistant apps to handle calculations on-the-go.

In [1]:
# Installing Langchain
!pip install -q -U langchain langchain_community langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.1 MB/s eta 0:00:00


In [2]:
# Importing userdata from Google Colab to securely store API keys
from google.colab import userdata
import os

gemini_api_key = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = 'GOOGLE_API_KEY'

# Importing GoogleGenAIChat Class from Google Gemini Langchain module
from langchain_google_genai import ChatGoogleGenerativeAI

# Optimizing model features
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key= gemini_api_key,
    temperature=0.7,
    max_tokens= 300,
    timeout=None,
    max_retries= 6,
)

In [3]:
ai_msg= llm.invoke("What is Tool Calling?")
print(ai_msg.content)

Tool calling, also sometimes referred to as **function calling** or **API calling**, is a powerful capability of large language models (LLMs) like GPT-4, Claude, and others. It allows the LLM to interact with external tools or APIs to perform actions, gather information, or extend its functionality beyond its training data.

Here's a breakdown of what tool calling is and how it works:

**Core Idea:**

Instead of just generating text, the LLM can now generate structured data that specifies:

1. **Which tool to use:** It identifies the appropriate tool or API based on the user's request.
2. **What parameters to pass:** It determines the necessary inputs for the chosen tool.

This structured data is then passed to an external system (the "tool") which executes the action and returns the result. The LLM can then use this result to inform its next response to the user.

**How it Works (Simplified):**

1. **User Input:** The user provides a prompt or request (e.g., "What's the weather in Lon

In [4]:
# Integrate the calculator with LangChain as a tool
from langchain.tools import Tool
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract a and b.

    Args:
        a: first int
        b: second int
    """
    return a - b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

@tool
def divide(a: int, b: int) -> int:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

llm_with_tools = llm.bind_tools([multiply, add, subtract, divide])

In [5]:
tools_list=[add, subtract, multiply, divide]
model_with_tools = llm.bind_tools(tools_list)

In [6]:
result = llm_with_tools.invoke("Hey how are you!")
print(result.content)

I'm doing great! Ready to perform some calculations. How can I help?


In [7]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a + b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)

In [8]:
#Initialize the LLM with tool-calling capabilities
from langchain.agents import initialize_agent, AgentType, AgentExecutor

# The calculator tool will handle basic arithmetic operations
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"


In [9]:
# Create an instance of the Calculator class
calculator = Calculator()

# Wrap the Calculator within a Tool object
# Provide a name, description and the function to be called
calculator_tool = Tool(
    name="Calculator",
    func=calculator.calculate,
    description="Useful for when you need to answer questions about math."
)

tools = [calculator_tool] # Use the Tool object in the tools list

In [14]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Initialize the Agent for Tool Calling
agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [13]:
#  Query the agent
response1 = agent.run("What is 10 - 20 + 15 x 10 / 2 ?")  # Uses the tool
print(response1)



> Entering new AgentExecutor chain...
I need to evaluate the expression 10 - 20 + 15 x 10 / 2 using the order of operations (PEMDAS/BODMAS). Multiplication and division have higher precedence than addition and subtraction, and are evaluated from left to right.
Action: Calculator
Action Input: 15 * 10 / 2
Observation: 75.0
Thought:Now I have 10 - 20 + 75. I will perform the subtraction and addition from left to right.
Action: Calculator
Action Input: 10 - 20
Observation: -10
Thought:Now I have -10 + 75.
Action: Calculator
Action Input: -10 + 75
Observation: 65
Thought:I now know the final answer
Final Answer: 65

> Finished chain.
65
